In [ ]:
import os
import pandas as pd

# read all recipes
base_path = './ingredient/'
files = os.listdir(base_path)

dfs = []
for file_name in files[1:]:
    dfs.append(pd.read_csv(os.path.join(base_path, file_name)))
ingredients = pd.concat(dfs, ignore_index=True)

effects = pd.read_csv('./categorize/effect.csv')

methods = pd.read_csv('./categorize/methods.csv')

categories = pd.read_csv('./categorize/categories.csv')

food_names = ingredients['Food_Name'].unique()

ingredients_category = pd.read_csv('./categorize/ingredient_category.csv')

In [3]:
base_path = './first_clean_recipe/'
files = os.listdir(base_path)

dfs = []

for file_name in files[1:]:
    dfs.append(pd.read_csv(os.path.join(base_path, file_name)))
steps = pd.concat(dfs, ignore_index=True)

In [ ]:
recipes = pd.DataFrame(columns=['Food_Name', 'Effects', 'Method', 'Category', 'Ingredients'])

for food_name in food_names:
    food_effects = list(effects[effects['Food_Name'] == food_name]['Effect'].values)
    method = methods[methods['Food_Name'] == food_name]['method'].values
    food_method = method[0] if len(method) > 0 else ''
    category = categories[categories['Food_Name'] == food_name]['category'].values
    food_category = category[0] if len(category) > 0 else ''
    food_ingredients = list(ingredients[ingredients['Food_Name'] == food_name]['Ingredient'].values)
    recipes = pd.concat([recipes, pd.DataFrame({'Food_Name': [food_name], 'Effects': [food_effects], 'Method': [food_method], 'Category': [food_category], 'Ingredients': [food_ingredients]})], ignore_index=True)

In [ ]:
entry_pug = '''tr
  .accordion-item
    h2.accordion-header
      button.accordion-button.collapsed(type='button' data-bs-toggle='collapse' data-bs-target='#collapse{id}' aria-expanded='false' aria-controls='collapse{id}')
        strong {recipe_name}
    #collapse{id}.accordion-collapse.collapse(data-bs-parent='#accordionExample')
      .accordion-body
        div
          .attributes
            {type}
            span.px-1 |
            {ingredients}
            span.px-1 |
            {effects}
          p {steps}
'''

type_tag = 'span.badge.rounded-pill.text-bg-primary {type}'

# also amount?
ingredient_tag = 'span.badge.rounded-pill.text-bg-dark {ingredient}'

effect_tag = 'span.badge.rounded-pill.text-bg-success {effect}'

checklist_pug = '''li
  .form-check
    input#ingredient_{id}.form-check-input(type='checkbox' value='')
    label.form-check-label.ml-3(for='ingredient_{id}') {category}
'''

In [ ]:
checks = []

for i, c in enumerate(ingredients_category['category'].unique()):
    c = ' '.join(c.split('_')).title()
    checks.append(checklist_pug.format(id=i, category=c))

with open('checks.pug', 'w') as f:
    f.write(''.join(checks))

# exec pug with generated tags
import subprocess
subprocess.run(['pug', 'checks.pug'], stderr=subprocess.DEVNULL)

In [ ]:
# for each recipe, generate a pug entry
entries = []

for i, r in recipes.iterrows():
    entry = {}
    recipe_name = r['Food_Name']
    type = r['Category']
    food_ingredients = [e.strip() for e in r['Ingredients']]
    food_effects = [e.strip() for e in r['Effects']]
    food_steps = steps[steps['Food_Name'] == recipe_name]['Steps'].values
    food_steps = food_steps[0] if len(food_steps) > 0 else ''
    food_steps = food_steps if food_steps == food_steps else ''
    # ingredient_tags = [ingredient_tag.format(ingredient=e) for e in food_ingredients]
    # effects_tags = [effect_tag.format(effect=e) for e in food_effects]
    # ingredient_tags = '\n            '.join(ingredient_tags)
    # effects_tags = '\n            '.join(effects_tags)
    type_tag = type_tag.format(type=type)
    entry['recipe_name'] = recipe_name
    entry['category'] = type
    entry['ingredients'] = food_ingredients
    entry['effects'] = food_effects
    entry['steps'] = food_steps
    entries.append(entry)
    # entry = entry_pug.format(id=i, recipe_name=recipe_name, type=type_tag, ingredients=ingredient_tags, effects=effects_tags, steps=food_steps)
    # entries.append(entry)

import json
with open('recipes.json', 'w', encoding='utf-8-sig') as f:
    json.dump(entries, f, ensure_ascii=False, indent=4)

# receipes_v2.json

In [17]:
import os
import pandas as pd

# read all recipes
base_path = './ingredient/'
files = os.listdir(base_path)

dfs = []
for file_name in files:
    if file_name == 'v1_JuanDiErShenXianFuShi.csv_ingredient.csv':
        continue
    dfs.append(pd.read_csv(os.path.join(base_path, file_name)))
ingredients = pd.concat(dfs, ignore_index=True)

effects = pd.read_csv('./categorize/effect.csv')

methods = pd.read_csv('./categorize/methods.csv')

categories = pd.read_csv('./categorize/categories.csv')

food_names = ingredients['Food_Name'].unique()

ingredients_category = pd.read_csv('./categorize/ingredient_category.csv')

In [18]:
base_path = './first_clean_recipe/'
files = os.listdir(base_path)

dfs = []

for file_name in files:
    if file_name == 'v1_JuanDiErShenXianFuShi.csv_ingredient.csv':
        continue
    dfs.append(pd.read_csv(os.path.join(base_path, file_name)))
steps = pd.concat(dfs, ignore_index=True)

In [19]:
recipes = pd.DataFrame(columns=['Food_Name', 'Effects', 'Method', 'Category', 'Ingredients'])

for food_name in food_names:
    food_effects = list(effects[effects['Food_Name'] == food_name]['Effect'].values)
    method = methods[methods['Food_Name'] == food_name]['method'].values
    food_method = method[0] if len(method) > 0 else ''
    category = categories[categories['Food_Name'] == food_name]['category'].values
    food_category = category[0] if len(category) > 0 else ''
    food_ingredients = list(ingredients[ingredients['Food_Name'] == food_name]['Ingredient'].values)
    recipes = pd.concat([recipes, pd.DataFrame({'Food_Name': [food_name], 'Effects': [food_effects], 'Method': [food_method], 'Category': [food_category], 'Ingredients': [food_ingredients]})], ignore_index=True)

In [20]:
# for each recipe, generate a pug entry
entries = []

for i, r in recipes.iterrows():
    entry = {}
    recipe_name = r['Food_Name']
    type = r['Category']
    food_ingredients = [e.strip() for e in r['Ingredients']]
    food_effects = [e.strip() for e in r['Effects']]
    food_method=[e.strip() for e in r['Method']]
    food_steps = steps[steps['Food_Name'] == recipe_name]['Steps'].values
    food_steps = food_steps[0] if len(food_steps) > 0 else ''
    food_steps = food_steps if food_steps == food_steps else ''
    # ingredient_tags = [ingredient_tag.format(ingredient=e) for e in food_ingredients]
    # effects_tags = [effect_tag.format(effect=e) for e in food_effects]
    # ingredient_tags = '\n            '.join(ingredient_tags)
    # effects_tags = '\n            '.join(effects_tags)
    # type_tag = type_tag.format(type=type)
    entry['recipe_name'] = recipe_name
    entry['category'] = type
    entry['ingredients'] = food_ingredients
    entry['effects'] = food_effects
    entry['steps'] = food_steps
    entry['methods'] = food_method
    entries.append(entry)
    # entry = entry_pug.format(id=i, recipe_name=recipe_name, type=type_tag, ingredients=ingredient_tags, effects=effects_tags, steps=food_steps)
    # entries.append(entry)

import json
with open('recipes_v2.json', 'w', encoding='utf-8-sig') as f:
    json.dump(entries, f, ensure_ascii=False, indent=4)